%matplotlib inline

In [ ]:
'''
  having trouble with jupyter so putting answer 1 here incase:
I do not think that introspection would be a good way to 
inform efforts to model AI for human cognition.  
I don't think it provides anything but a very broad layer.  
Although one could possible lay the ground work based on introspection, 
it wouldn't really provide anything that wasn't already there.  
It wouldn't help create new algorithms, or 
scientific data to help model the AI.  So, overall I don't think 
it would be helpful.


'''

Hello


In [ ]:
'''
    tsp.py solves the traveling sales person problem using a local search.
    for class cs344
@author austin gibson
@version feb 22, 2019

'''

from search import Problem, hill_climbing, simulated_annealing, \
    exp_schedule
from random import randrange


class TravelingSalesPerson(Problem):

    def __init__(self, initial, distances):
        self.initial = initial
        self.distances = distances

    # list of possible swaps.  Never swap first or last
    def actions(self, state):
        actions = []

        for i in range(1,6):
            j = randrange(1, 6)
            while j == i:
                j = randrange(1,6)
            actions.append([i,j])

        return actions

    # swap actions from state to new state
    def result(self, state, action):
        new_state = state[:]
        temp_state = new_state[action[0]]
        new_state[action[0]] = new_state[action[1]]
        new_state[action[1]] = temp_state
        return new_state

    # total distance of traveled cities gives value
    def value(self, state):
        totalDistance = 0
        for i in range(len(state)):
            if i + 1 < len(state):
                city1 = state[i]
                city2 = state[i + 1]
                totalDistance += self.distances[(city1, city2)]
        return -totalDistance


if __name__ == '__main__':

    cities = ["Kings Landing", "Riverrun", "Winterfell", "Braavos", "Volantis", "Oldtown", "Kings Landing"]

    distances = {
        ("Kings Landing", "Braavos"): 22,
        ("Kings Landing", "Winterfell"): 5,
        ("Kings Landing", "Riverrun"): 8,
        ("Kings Landing", "Volantis"): 13,
        ("Kings Landing", "Oldtown"): 20,
        ("Braavos", "Kings Landing"): 22,
        ("Winterfell", "Kings Landing"): 5,
        ("Riverrun", "Kings Landing"): 8,
        ("Volantis", "Kings Landing"): 13,
        ("Oldtown", "Kings Landing"): 20,
        ("Braavos", "Winterfell"): 13,
        ("Braavos", "Riverrun"): 10,
        ("Braavos", "Volantis"): 14,
        ("Braavos", "Oldtown"): 12,
        ("Winterfell", "Braavos"): 13,
        ("Riverrun", "Braavos"): 10,
        ("Volantis", "Braavos"): 14,
        ("Oldtown", "Braavos"): 12,
        ("Winterfell", "Riverrun"): 8,
        ("Winterfell", "Volantis"): 7,
        ("Winterfell", "Oldtown"): 18,
        ("Riverrun", "Winterfell"): 8,
        ("Volantis", "Winterfell"): 7,
        ("Oldtown", "Winterfell"): 18,
        ("Riverrun", "Volantis"): 13,
        ("Riverrun", "Oldtown"): 6,
        ("Volantis", "Riverrun"): 13,
        ("Oldtown", "Riverrun"): 6,
        ("Volantis", "Oldtown"): 15,
        ("Oldtown", "Volantis"): 15
    }

    p = TravelingSalesPerson(cities, distances)
    print('Initial                       ' + str(cities)
          + '\tvalue: ' + str(p.value(cities)))

    hill_solution = hill_climbing(p)
    print('Hill-climbing solution        ' + str(hill_solution)
          + '\tvalue: ' + str(p.value(hill_solution))
          )

    annealing_solution = simulated_annealing(
        p,
        exp_schedule(k=20, lam=0.005, limit=1000)
    )
    print('Simulated annealing solution  ' + str(annealing_solution)
          + '\tvalue: ' + str(p.value(annealing_solution))
          )
    

Initial                       ['Kings Landing', 'Riverrun', 'Winterfell', 'Braavos', 'Volantis', 'Oldtown', 'Kings Landing']	value: -78
Hill-climbing solution        ['Kings Landing', 'Riverrun', 'Winterfell', 'Volantis', 'Braavos', 'Oldtown', 'Kings Landing']	value: -69
Simulated annealing solution  ['Kings Landing', 'Winterfell', 'Volantis', 'Braavos', 'Oldtown', 'Riverrun', 'Kings Landing']	value: -52


In [ ]:
'''
    schedule.py solves the problem of schedule courses in homework1
    - based on Zebra()
    @author austin gibson
    @version Feb 23, 2019
'''

from csp import CSP, min_conflicts, backtracking_search, AC3, parse_neighbors
from search import depth_first_graph_search

def Schedule():

    variables = 'cs108 cs112 cs212 cs214 cs232 cs262'.split()
    courses = ["cs108", "cs112", "cs212", "cs214", "cs232", "cs262"]
    TimeSlot = 'mwf900 tth1030 mwf1130 tth130 mwf130'.split()
    Rooms = 'nh253 sb382'.split()
    Assignments = {
        "cs108": "adams",
        "cs112": "vanderlinden",
        "cs212": "bailey",
        "cs214": "schuurman",
        "cs232": "adams",
        "cs262": "bailey"
    }

    #variables = Courses + Faculty + TimeSlot + Rooms
    #loop through courses and make course-possibleVal triples
    domains = {}
    for var in variables:
        domains[var] = []
        faculty = Assignments[var]
        for time in TimeSlot:
            for room in Rooms:
                domains[var].append([faculty, time, room])
    """
    neighbors = parse_neighbors(cs108:cs112; cs108:cs212; cs108:cs214; cs108:cs232; cs108:cs262;
                        cs112:cs212; cs112:cs214; cs112:cs232; cs112:cs262;
                        cs212:cs214; cs212:cs232; cs212:cs262;
                        cs214:cs232; cs214:cs262; cs232:cs262; , variables)
    """

    neighbors = {}

    for i in range(len(courses)):
        neighborslist = []
        for j in range(i + 1, len(courses)):
            neighborslist.append(courses[j])
        neighbors[courses[i]] = neighborslist

    def schedule_constraint(A, a, B, b, recurse=0):
        #checks time/faculty constraint
        if A[0] == B[0] and A[1] == A[1]:
            return False
        #checks time/room
        if A[1] == B[1] and A[2] == A[2]:
            return False
        return True

    return CSP(variables, domains, neighbors, schedule_constraint)


def print_solution(result):
    variables = 'cs108 cs112 cs212 cs214 cs232 cs262'.split()
    for h in variables:
        print('Course', h)
        for (var, val) in result.items():
            if val == h: print('\t', var)


puzzle = Schedule()


#result = depth_first_graph_search(puzzle)
#result = AC3(puzzle)
#result = backtracking_search(puzzle)
result = min_conflicts(puzzle, max_steps=1000)

if puzzle.goal_test(puzzle.infer_assignment()):
    print("Solution:\n")
    print_solution(result)
else:
    print("failed...")
    print(puzzle.curr_domains)
    puzzle.display(puzzle.infer_assignment())